In [11]:
import os
import re
from bs4 import BeautifulSoup
import pandas as pd

def list_files(path) -> pd.DataFrame: 
    dirs = os.listdir(f'{path}/juris')

    file_paths = []
    years = []
    ids = []
    for dir in dirs:
        files = os.listdir(f'{path}/juris/{dir}')
        for file in files:
            # append current year which is the current dir name
            years.append(dir)

            # append identifier of file
            ids.append(file)

            # build and append file name
            file_paths.append(f'{path}/juris/{dir}/{file}')

    return pd.DataFrame({'identifier': ids, 'year': years, 'file_path': file_paths})

# get the header

def match_keywords():
    jargon__df = pd.read_csv('./jargon.csv', index_col=0)
    print(jargon__df)




juris__df = list_files(os.getcwd())
print(juris__df.head())


                  identifier       year  \
0   1259024318844998577.html  1901-1920   
1   1259025995477516486.html  1901-1920   
2  12590286321646095756.html  1901-1920   
3   1259030844961946103.html  1901-1920   
4  12590448821684625643.html  1901-1920   

                                           file_path  
0  d:\Projects\To Github\LaRJ-Corpus\raw jurispru...  
1  d:\Projects\To Github\LaRJ-Corpus\raw jurispru...  
2  d:\Projects\To Github\LaRJ-Corpus\raw jurispru...  
3  d:\Projects\To Github\LaRJ-Corpus\raw jurispru...  
4  d:\Projects\To Github\LaRJ-Corpus\raw jurispru...  


In [12]:
juris__df.head()

,identifier,year,file_path
0,1259024318844998577.html,1901-1920,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
1,1259025995477516486.html,1901-1920,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
2,12590286321646095756.html,1901-1920,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
3,1259030844961946103.html,1901-1920,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
4,12590448821684625643.html,1901-1920,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...


In [13]:
juris__df.tail()

,identifier,year,file_path
66590,1615183088809149243.html,2021,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
66591,1615268588459747150.html,2021,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
66592,161594165740000218.html,2021,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
66593,1615953315892765798.html,2021,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...
66594,16159550661388784210.html,2021,d:\Projects\To Github\LaRJ-Corpus\raw jurispru...


# extract the words out of the html file

In [14]:
sample_file = juris__df.iloc[-1].to_numpy()
sample_file_path = 2
with open(sample_file[sample_file_path]) as file:
    document = file.read()
    dom = BeautifulSoup(document, 'lxml')

sample_text = dom.get_text()
print(sample_text)
print(len(sample_text))




G.R. No. 204152, January 19, 2021




EN BANC

				G.R. No. 204152, January 19, 2021
DENR EMPLOYEES UNION (DENREU) AND KALIPUNAN NG MGA KAWANI SA KAGAWARAN NG KALIKASAN (K4), PETITIONERS, VS. SECRETARY FLORENCIO B. ABAD OF THE DEPARTMENT OF BUDGET AND MANAGEMENT AND THE COMMISSION ON AUDIT, RESPONDENTS.D E C I S I O N

PERALTA, C.J.:
Petitioners Department of Environment and Natural Resources (DENR) Employees Union (DENREU) and Kalipunan ng mga Kawani sa Kagawarang Kalikasan (K4) filed the present Petition for Injunction and Prohibition under Rule 65 of the Rules of Court assailing the constitutionality of Budget Circular No. 2011-5 dated December 26, 2011 of the Department of Budget and Management (DBM). The circular placed a P25,000.00 ceiling on the amount of the Collective Negotiation Agreement (CNA) Incentive for 2011. In compliance therewith, respondent Commission on Audit (COA) disallowed the DENR's issuance of CNA Incentives to its officers and employees for exceeding said l

Jurisprudence document clearly has 12 pages and 8000+ words in one example. How much more for other documents? We need to impose a limit in such a way that GPT-3.5 can still take in these text without having to break its limit of 4096 tokens or roughly 1500 words

In [15]:
def identify_text(juris__df) -> pd.DataFrame:
    # open all files extract text with a certain limit
    for itn, file_path in enumerate(juris__df.loc[:, 'file_path']):
        print(file_path)
        with open(file_path) as file:
            document = file.read()
            dom = BeautifulSoup(document, 'lxml')

            # extract text of html file
            text = dom.get_text()

            if itn % 100 == 0:
                pass
                # print(text)

            # make api call with GPT 3.5 here

            # close file
            file.close()
            

# content__df = identify_text(juris__df)


# GPT-3.5 API
feed into chat gpt with the ff prompt
Is this a labor related jurisprudence
answer only yes or no

In [16]:
import openai
from openai import error
import requests
import time
import random
import func_timeout


# Check and load OpenAI API key

In [27]:
# ff. imports are for getting secret values from .env file
import os
from dotenv import load_dotenv
from pathlib import Path

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(os.getcwd()).resolve().parent
print(BASE_DIR)

# specify path of .env file in which to access the secret values from
# which is teh BASE_DIR where manage.py and requirements.txt live, 
# then append the string .env using os.path.join()
load_dotenv(os.path.join(BASE_DIR, '.env'))

D:\Projects\To Github\LaRJ-Corpus


True

In [18]:
# Load your API key from an environment variable or secret management service
OPEN_AI_API_KEY = os.environ['OPEN_AI_API_KEY']
auth_token = f"Bearer {OPEN_AI_API_KEY}"
openai.api_key = OPEN_AI_API_KEY

In [19]:
# # test gpt-3.5 model 
# response = requests.post('https://api.openai.com/v1/chat/completions', headers={
#     "Content-Type": "application/json",
#     "Authorization": auth_token
# }, data={
#     "model": "gpt-3.5-turbo",
#     "messages": [{"role": "user", "content": "hello!"}]
# })

# response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "hello!"}])
# print(response)

Testing GPT 3.5 to identify whether or not jurisprudence text is labor related or not labor related

In [20]:
# prompt = f"""{sample_text[:18000]}

# Identify if this case law is labor related answer in only binary terms. whether "labor  related" or "not labor related"""

# print(prompt)

In [21]:
# response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
# print(response['choices'][0]['message']['content'])

In [22]:
# # open random jurisprudence file
# sample_file_2 = juris__df.iloc[random.randint(0, 60000)].to_numpy()
# sample_file_path_2 = 2
# with open(sample_file_2[sample_file_path_2]) as file:
#     document = file.read()
#     dom = BeautifulSoup(document, 'lxml')

# sample_text_2 = dom.get_text()
# print(sample_text_2)
# print(len(sample_text_2))

# # create prompt with this random jurisprudence file
# prompt_2 = f"""{sample_text_2[:17000]}

# Read and identify carefully whether this piece of case law is labor related or not labor related. Answer only in binary terms "labor related" or "not labor related"""
# print(prompt_2)

# # feed prompt to chat gpt
# response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt_2}])
# answer = response['choices'][0]['message']['content']
# print(f'response: {answer}')


# Do this for all 60000+ files
using identify_text()

In [23]:
def send_prompt(prompt, model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(model=model, messages=[{"role": "user", "content": prompt}])
    answer = response['choices'][0]['message']['content']
    return answer

def identify_text_2(juris__df) -> pd.DataFrame:

    # populate list with all the answers of GPT 3.5
    answers = []
    accepted_file_paths = []

    # open all files extract text with a certain limit
    for itn, file_path in enumerate(juris__df.loc[:, 'file_path']):
        print(file_path)
        try:
            with open(file_path) as file:
                document = file.read()
                dom = BeautifulSoup(document, 'lxml')

                # extract text of html file
                text = dom.get_text()

                # make api call with GPT 3.5 here

                # create prompt with this random jurisprudence file
                prompt = f"""{text[:17000]}

                Read and identify carefully whether this piece of case law is labor related or not labor related. Answer only in binary terms "labor related" or "not labor related"""
                # print(prompt)

                # feed prompt to chat gpt to helper function func timeout to catch
                # unresponsive rqeuests or requests that take to long. Will raise error if
                # it takes too long
                answer = func_timeout.func_timeout(40, send_prompt, args=(prompt,))

                # append response if successful
                # print(f'response: {answer}')
                answers.append(answer)
                accepted_file_paths.append(file_path)
                
                # close file
                file.close()
        except error.Timeout:
            print('server has timed out retrying prompt')

            # record what answers have been generated before error
            # record what file path did the error occured
            # pd.DataFrame({'file_path': accepted_file_paths,'answer': answers})
            answers.append('TIMEOUT_ERROR')
            accepted_file_paths.append(file_path)

        except error.RateLimitError:
            print('Rate limit reached for default-gpt-3.5-turbo in organization org-lQNgFbb5falHwdZD3HS05eya on requests per min. Limit: 20 / min. Please try again in 3s')
            
            # record what answers have been generated before error
            # record what file path did the error occured
            # pd.DataFrame({'file_path': accepted_file_paths,'answer': answers})
            answers.append('RATE_LIMIT_ERROR')
            accepted_file_paths.append(file_path)

            # initiate 5 second delay
            print('5 second delay occuring')
            time.sleep(5.0)

        except error.ServiceUnavailableError as e:
            print(e._message)

            # pd.DataFrame({'file_path': accepted_file_paths,'answer': answers})
            answers.append('SERVICE_UNAVAILABLE_ERROR')
            accepted_file_paths.append(file_path)

        except error.InvalidRequestError as e:
            print(e._message)

            # pd.DataFrame({'file_path': accepted_file_paths,'answer': answers})
            answers.append('INVALID_REQUEST_ERROR')
            accepted_file_paths.append(file_path)

        except func_timeout.FunctionTimedOut:
            print('Request took too long will move to next file')

            # pd.DataFrame({'file_path': accepted_file_paths,'answer': answers})
            answers.append('TIME_LIMIT_REACHED')
            accepted_file_paths.append(file_path)

        except error.APIConnectionError:
            print('Error communicating with OpenAI')

            # 
            answers.append('API_CONNECTION_ERROR')
            accepted_file_paths.append(file_path)

        finally:
            print('Moving on. Waiting for 5 seconds to avoid traffic')
            answers_partial = pd.DataFrame({'file_path': accepted_file_paths,'answer': answers})
            answers_partial.to_csv('./answers__partial.csv')
            time.sleep(5.0)


    return pd.DataFrame({'file_path': accepted_file_paths,'answer': answers})
            




In [24]:
# answers__df = identify_text_2(juris__df)

## save juris__df as .csv

In [25]:
# juris__df.to_csv('./juris__df.csv')